In [1]:
import logging
import os
from typing import Iterator, Optional
import openai

from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.callbacks.base import CallbackManager
from langchain.chat_models import ChatOpenAI
import chainlit as cl
from dotenv import load_dotenv

from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.chat_engine.types import BaseChatEngine, ChatMode

from llama_index import (
    LLMPredictor,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
    set_global_service_context,
)
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

In [2]:
# obtain gpt model name from environment variables
gpt_model = "gpt-4"
gpt_temperature = 0.4

In [3]:
document_list = [
	'../quant_scraper/docs/vbt_pro/cookbook.md',
	# '../quant_scraper/docs/vbt_pro/documentation.md',
]

## DefaultVectorStorage

In [4]:
def build_default_vector_storage():
		print('Loading documents...')
		documents = SimpleDirectoryReader(input_files=document_list).load_data()
		print('Building index...')
		index = GPTVectorStoreIndex.from_documents(documents, show_progress=True)

		llm_predictor = LLMPredictor(
			llm=ChatOpenAI(
				temperature=gpt_temperature,
				model_name=gpt_model,
				max_tokens=2048,
				streaming=True,
			),
		)

		service_context = ServiceContext.from_defaults(
			llm_predictor=llm_predictor,
			chunk_size=1024,
			callback_manager=CallbackManager([cl.LlamaIndexCallbackHandler()]),
		)

		retriever = VectorIndexRetriever(
			index=index,
			similarity_top_k=10,
		)
		# configure response synthesizer
		response_synthesizer = get_response_synthesizer(
			response_mode="tree_summarize", service_context=service_context)

		# assemble query engine
		query_engine = RetrieverQueryEngine.from_args(
			streaming=True,
			retriever=retriever,
			response_synthesizer=response_synthesizer,
			service_context=service_context,
			node_postprocessors=[
				SimilarityPostprocessor(similarity_cutoff=0.7)
			]
		)

		return query_engine, index, service_context

In [5]:
# build default vector storage
# query_engine_1, index_1, service_context_1 = build_default_vector_storage()

In [6]:
test_query = """
In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?
"""

# query_engine_1.query(test_query)